In [ ]:
dataset= 'Adult'  #Name of the dataset from ['Adult','Bank']
order= 'random'  # Order of data points with respect to protected group - random or arbitrary

Ktarget = 10   #Target or desired number of clusters 


#Point 1. Capture all costs as squared euclidean 

In [ ]:
import numpy as np
import math
import pandas as pd
from sklearn.utils import shuffle
import random
import numba
from numba import jit , njit
from csv import writer
import time
from scipy.spatial import distance
#!pip install scikit-learn-extra
from sklearn.cluster import KMeans
import itertools
import matplotlib.pyplot as plt

In [ ]:
def load_dataset(dataset,Ktarget):
    
    flag= True
    if dataset=='Adult':
        _filename = 'adult_p.csv'
        OriCenters=[]
    elif dataset=='Bank':
        _filename = 'bank_p.csv'
        OriCenters=[]
        
    elif dataset=='Synthetic-1d':
        
        OriCenters=[1]
        cluster1 = np.random.normal([1], 2, 1000)
        
            
        df=pd.DataFrame(cluster1, columns=['X'])
        for kk in range(1,Ktarget):
           
            cluster1 = np.random.normal([1+7*kk],2 , 1000)
            OriCenters.append([1+7*kk])
           
          
            df1=pd.DataFrame(cluster1, columns=['X'])
            df=pd.concat([df,df1])
            
      
        df = df.reset_index()  
        
        df = df.drop(df.columns[0],axis=1)
       
        df=shuffleDataset(df)
        OriCenters=np.array(OriCenters)
        flag= False
        
        
    elif dataset=='Synthetic-2d-over':
        

        cluster1 = np.random.multivariate_normal([1,3], [[1, 0], [0, 1]], 1000)
        OriCenters=[[1,3]]
            
        df=pd.DataFrame(cluster1, columns=['X','Y'])
        for kk in range(1,Ktarget):
           
            cluster1 = np.random.multivariate_normal([1+5*kk,3], [[1, 0], [0, 1]], 1000)
            
            OriCenters.append([1+5*kk,3])
          
            df1=pd.DataFrame(cluster1, columns=['X','Y'])
            df=pd.concat([df,df1])
            
      
        df = df.reset_index()
        df = df.drop(df.columns[0],axis=1)
        #print(df)
        
        
        plot2Ddata(df['X'],df['Y'])
        df=shuffleDataset(df)
        OriCenters=np.array(OriCenters)
        flag= False
    
        
        
        
    elif dataset=='Synthetic-2d': #well separated
        

        cluster1 = np.random.multivariate_normal([1,3], [[1, 0], [0, 1]], 1000)
        OriCenters=[[1,3]]
            
        df=pd.DataFrame(cluster1, columns=['X','Y'])
        for kk in range(1,Ktarget):
           
            cluster1 = np.random.multivariate_normal([1+7*kk,3], [[1, 0], [0, 1]], 1000)
            
            OriCenters.append([1+7*kk,3])
          
            df1=pd.DataFrame(cluster1, columns=['X','Y'])
            df=pd.concat([df,df1])
            
      
        df = df.reset_index()
        df = df.drop(df.columns[0],axis=1)
        #print(df)
        
        
        plot2Ddata(df['X'],df['Y'])
        df=shuffleDataset(df)
        OriCenters=np.array(OriCenters)
        flag= False
        
    elif dataset=='Synthetic-1d-Over':
        
        OriCenters=[1]
        cluster1 = np.random.normal([1], 2, 1000)
        
            
        df=pd.DataFrame(cluster1, columns=['X'])
        for kk in range(1,Ktarget):
           
            cluster1 = np.random.normal([1+5*kk],2 , 1000)
            OriCenters.append([1+5*kk])
           
          
            df1=pd.DataFrame(cluster1, columns=['X'])
            df=pd.concat([df,df1])
            
      
        df = df.reset_index()  
        
        df = df.drop(df.columns[0],axis=1)
       
        df=shuffleDataset(df)
        OriCenters=np.array(OriCenters)
        flag= False
        
    elif dataset=='Synthetic-arb':
        flag= False
        pass
    
    if flag:
        
        df = pd.read_csv(_filename, sep=',')#[:10000]
        df= df.round(decimals=5)
        df = df.dropna()
    
        if order=='random':
            df=shuffleDataset(df)
        
    print('Dataset loaded........')
    
    #df.to_csv('SeededData/'+str(dataset)+'_seed_'+str(seed)+'.csv',index=False)
    
    return df,OriCenters
        
def plot2Ddata(x,y):
    plt.plot(x, y, 'x')
    plt.axis('equal')
    plt.show()

In [ ]:
def shuffleDataset(df):

    df = shuffle(df)
    df = shuffle(df)    #random with respect to group and distance
    df = shuffle(df)
    df = df.reset_index()
    df = df.drop(columns='index')
    return df
    
    

In [ ]:
@njit(parallel=False)
def EuclideanDistance(x,y):
    sum_ = 0
    
    for i in range(0,len(x)):
        
        sum_ = sum_ + (x[i]-y[i])**2

    return sum_#**0.5


def EuclideanDistance_(x,y):
    sum_ = 0
#     print(x)
#     print(y)
#     print()
    
    for i in range(0,len(x)):
        
        sum_ = sum_ + (x[i]-y[i])**2

    return sum_#**0.5


In [ ]:
@njit(parallel=False)
def Distances(rows,center): 
    # Numpy array - center and rows
    #Finds the distance of rows to 1 center in fast parallel way
    
    #need centers in col
    rr = len(rows)
    cc = len(center) 
   
    dist=np.zeros((rr, cc)) 
    
    
    
    for indr in range(0,len(rows)):
        
        
        
        for indce in range(0,len(center)):
            
            dis = EuclideanDistance(rows[indr],center[indce])
            dist[indr][indce]= dis
            
    
    return dist
 
#@njit(parallel=False)
def sortedDistances1(centers,row): 
    # Numpy array - center and rows
    #Finds the distance of rows to 1 center in fast parallel way
    
    distOfCenters = []
    
    
    for indce in range(0,len(centers)): # each center is tuple of index, center vector 
        dis = EuclideanDistance_(row,centers[indce][1])
        distOfCenters.append(tuple((dis,centers[indce][0])))  #add distance, original center id 
        
            
    #sorted_centers = sorted(distToCenters, key=lambda x: x[0])
    distOfCenters.sort()  #inPlace sorts 
    
    
    return distOfCenters
            
   

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
random.seed(0)
np.random.seed(0)

def kmedian(x,k, no_of_iterations):
    idx = np.random.choice(len(x), k, replace=False)
    #Randomly choosing Centroids
    centroids = x[idx, :] #Step 1
     
    #finding the distance between centroids and all the data points
    distances = cdist(x, centroids ,'sqeuclidean') #Step 2
     
    #Centroid with the minimum Distance
    points = np.array([np.argmin(i) for i in distances]) #Step 3
     
    #Repeating the above steps for a defined number of iterations
    #Step 4
    for _ in range(no_of_iterations):
        centroids = []
        for idx in range(k):
            #Updating Centroids by taking mean of Cluster it belongs to
            assign = x[points==idx]
            temp_cent = np.median(assign,axis=0)
            centroids.append(temp_cent)
 
        centroids = np.vstack(centroids) #Updated Centroids
         
        distances = cdist(x, centroids ,'sqeuclidean')
        points = np.array([np.argmin(i) for i in distances])
   
    cost = 0
        
    for i in range(0,len(distances)):
        cost += distances[i][points[i]]
            
    return points,cost


In [ ]:
def CapHeur(Ktarget,X,capacities):
    
    n_row, n_col = X.shape
    rand_indices = np.random.choice(n_row, size = Ktarget)
    maxiter=100
  
    centroids = X[rand_indices]

    
   
    for itr in range(maxiter):
        distances_to_centroids = Distances(X, centroids)#distance.cdist(X, centroids, 'sqeuclidean')
        

        
        cluster_assignment=[]
        
        capacities_iter= np.array(capacities,dtype=np.int64)#(tuple of form (center idx,capacity))
        #Deep copies it into this np.array()
        
        for idx in range(0,len(X)):
            #print(distances_to_centroids[idx])
            sorted_centerIDs = np.argsort(distances_to_centroids[idx])
            
            for idc in sorted_centerIDs:
            
                if capacities_iter[int(idc)][1] >0:
                    cluster_assignment.append(idc)
                    capacities_iter[int(idc)][1] -=1
                    break
                    
                    
        #print("Found {} assignements for iteration {}.".format(len(cluster_assignment), itr))
               
        cluster_assignment=np.array(cluster_assignment)   
        
        new_centroids=[]
        for i  in range(Ktarget):
            
            assign= np.array([X[cluster_assignment == i]])
            #print('shape of assign is ',assign)
            med = np.median(assign,axis=1)
            #print('shape of med is ',med)
            new_centroids.append(med)
            
            
       
            
        new_centroids = np.array(new_centroids)
        new_centroids = new_centroids.reshape(Ktarget,len(X[0]))
        
            
        if np.all(centroids == new_centroids):
                break
        
        centroids = new_centroids
        #print('centroids are ',centroids)
            
    
    
    cost = []
    for i in range(Ktarget):
        cluster_data = X[cluster_assignment == i]
        distances = distance.cdist(cluster_data, [centroids[i]], 'euclidean')
        cost.append( np.sum(distances ** 2))
    
    return np.sum(cost)


def LowerBoundCapHeuristic(dfValues):
    ans_mean=[]
    ans_std=[]
    KtargetList =[1099,844,1231,1354,1488,292,474,806,1153,3143,4657,5946,7136,9865]
    for Ktarget in KtargetList:
        print('K is ',Ktarget)
        w_list = []
        seeds=[0,100,200]#,300,400,500,600,700,800,900]
        for seed in seeds:
            random.seed(seed)
            np.random.seed(seed)

            capacities=[]

            for i in range(Ktarget):
                capacities.append([i,np.ceil(len(dfValues)/Ktarget)])  #(centerId,capacity)

            w_list.append(CapHeur(Ktarget,dfValues,capacities))

        print(np.mean(w_list))
        print(np.std(w_list))
        print('*'*10)
        ans_mean.append(np.mean(w_list))
        ans_std.append(np.std(w_list))
    return ans_mean,ans_std

def CostVanillaKmedian1(dfValues):
    
    ans_mean=[]
    ans_std = []

    KtargetList =[1231,1354,1488,292,474,806,1153,3143,4657,5946,7136,9865]
    for Ktarget in KtargetList:
        print('K is ',Ktarget)
        seeds=[0,500]#,300,400,500,600,700,800,900]
        cost = []
        
        for seed in seeds:
            random.seed(seed)
            np.random.seed(seed)
            _,kmean_obj = kmedian(dfValues, Ktarget, 100) #KMeans(n_clusters=Ktarget, init='random' ,random_state=seed).fit(dfValues)

            cost.append(kmean_obj)    #Sum of squarred euc dis 


        print(np.mean(cost))
        print(np.std(cost))
        print('*'*10)
        ans_mean.append(np.mean(cost))
        ans_std.append(np.std(cost))
        
        
    return ans_mean,ans_std#, np.std(cost)    #average over 10 runs     



df,OriCenters = load_dataset(dataset,10)
dfValues = df.values

df,OriCenters = load_dataset(dataset,10)
dfValues = df.values
ans_mean,ans_std=CostVanillaKmedian1(dfValues)

print(ans_mean)
print(ans_std)
print()


ans_mean,ans_std=LowerBoundCapHeuristic(dfValues)
print()
print()

print(ans_mean)
print(ans_std)
print()


In [ ]:
#Utility functions

def CostVanillaKmeanspp(Ktarget,dfValues):
    #k-means++

    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='k-means++' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
    return np.mean(cost), np.std(cost)     #average over 10 runs  
 
    
def CostVanillaKmeansList(Ktarget,dfValues):
    
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='random' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
        labels = kmean_obj.labels_ 
        
        cluster_centers = kmean_obj.cluster_centers_
        
        cost_seed = []
        for i in range(Ktarget):
            cluster_data = dfValues[labels == i]
            distances = distance.cdist(cluster_data, [cluster_centers[i]], 'euclidean')
            cost_seed.append( np.sum(distances ** 2))
        cost.append(np.max(cost_seed))
        
        
    
        
    
    
    return cost#, np.std(cost)    #average over 10 runs     

def CostVanillaKmeans(Ktarget,dfValues):
    
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        kmean_obj = KMeans(n_clusters=Ktarget, init='random' ,random_state=seed).fit(dfValues)
        
        cost.append(kmean_obj.inertia_)    #Sum of squarred euc dis 
        
    
    
    return np.mean(cost)#, np.std(cost)    #average over 10 runs     


def CostVanillaKrandom(Ktarget,dfValues):
    
    seeds=[0,100,200,300,400,500,600,700,800,900]
    cost = []
    for seed in seeds:
        random.seed(seed)
        np.random.seed(seed)
        
        n_row, n_col = dfValues.shape
        rand_indices = np.random.choice(n_row, size = Ktarget)
        centroids = dfValues[rand_indices]
        
        distances_to_centroids = distance.cdist(dfValues, centroids, 'sqeuclidean')
        cluster_assignment = np.argmin(distances_to_centroids, axis = 1)
        heterogeneity=0
        for i in range(Ktarget):
            cluster_data = dfValues[cluster_assignment == i]
            distances = distance.cdist(cluster_data, [centroids[i]], 'euclidean')
            heterogeneity += np.sum(distances ** 2)
        
        cost.append(heterogeneity)
    
    return np.mean(cost)#, np.std(cost)